In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import MaxPooling2D, Dropout, Flatten,Input,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer



In [2]:
trn_dtgn = ImageDataGenerator(
    rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

trn_gnrtr = trn_dtgn.flow_from_directory('Dataset/',target_size=(224,224),batch_size=16,class_mode='categorical')


Found 1125 images belonging to 3 classes.


In [3]:
baseModel = VGG16(weights="imagenet",include_top=False,input_tensor=Input(shape=(224,224,3)))

In [11]:
for layer in baseModel.layers:
    layer.trainable = False

head = baseModel.output
head = MaxPooling2D(pool_size=(3,3))(head)
head = Flatten(name="flatten")(head)
head = Dense(64,activation= 'relu')(head)
head = Dropout(0.6)(head)
head = Dense(16,activation='relu')(head)
head = Dense(3,activation='softmax')(head)

Tmodel = Model(inputs=baseModel.input,outputs=head)

In [13]:
opt = Adam(lr=0.001,decay=(0.001/25))
Tmodel.compile(loss="binary_crossentropy",optimizer=opt,metrics=['accuracy'])


In [14]:
Tmodel.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [15]:
Tmodel.fit(trn_gnrtr,steps_per_epoch=5,epochs=20)

Epoch 1/20
5/5 [==============================] - 41s 8s/step - loss: 0.8036 - accuracy: 0.4500
Epoch 2/20
5/5 [==============================] - 40s 8s/step - loss: 0.8866 - accuracy: 0.3500
Epoch 3/20
5/5 [==============================] - 39s 8s/step - loss: 0.6545 - accuracy: 0.4500
Epoch 4/20
5/5 [==============================] - 38s 8s/step - loss: 0.6068 - accuracy: 0.4875
Epoch 5/20
5/5 [==============================] - 40s 8s/step - loss: 0.6226 - accuracy: 0.4375
Epoch 6/20
5/5 [==============================] - 39s 8s/step - loss: 0.6152 - accuracy: 0.5125
Epoch 7/20
5/5 [==============================] - 39s 8s/step - loss: 0.6046 - accuracy: 0.4875
Epoch 8/20
5/5 [==============================] - 44s 9s/step - loss: 0.5931 - accuracy: 0.5125
Epoch 9/20
5/5 [==============================] - 40s 8s/step - loss: 0.5259 - accuracy: 0.6250
Epoch 10/20
5/5 [==============================] - 40s 8s/step - loss: 0.6219 - accuracy: 0.4625
Epoch 11/20
5/5 [======================

In [17]:
Tmodel.save("xraymodel.h5")